In [1]:
import pandas as pd
import io
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
print ('Ran')

Ran


In [2]:
# app_test data features
app_test = pd.read_csv('F:/Project/Merged_train_test/merged_test.csv')
print('app_test data shape: ', app_test.shape)
app_test.head()

app_test data shape:  (48744, 374)


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,...,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0.0,0.0,31.0,1.0,0.0,0.0,0.0,0.0,110.0,30.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0.0,0.0,79.0,7.0,0.0,0.0,0.0,0.0,103.0,41.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0.0,0.0,164.0,0.0,0.0,0.0,0.0,0.0,263.0,133.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
app_test[['DAYS_EMPLOYED',
        'DAYS_FIRST_DRAWING',
        'DAYS_FIRST_DUE',
        'DAYS_LAST_DUE_1ST_VERSION',
        'DAYS_LAST_DUE',
        'DAYS_TERMINATION']].replace(365243, np.nan, inplace=True)

In [10]:
def replace_XNA_XAP(table):

    # Replace all values of 'XNA', 'XAP' with np.nan
    table.replace(to_replace = {'XNA': np.nan, 'XAP': np.nan}, value = None, inplace = True)
    
    return table

In [11]:
app_test = replace_XNA_XAP(app_test)

In [41]:
XNA_col=[col for col in app_test.columns if '_XNA' in col]

In [42]:
XAP_col=[col for col in app_test.columns if '_XAP' in col]

In [46]:
XNA_col

['NAME_CONTRACT_TYPE_XNA',
 'NAME_CASH_LOAN_PURPOSE_XNA',
 'NAME_PAYMENT_TYPE_XNA',
 'CODE_REJECT_REASON_XNA',
 'NAME_CLIENT_TYPE_XNA',
 'NAME_GOODS_CATEGORY_XNA',
 'NAME_PORTFOLIO_XNA',
 'NAME_PRODUCT_TYPE_XNA',
 'NAME_SELLER_INDUSTRY_XNA',
 'NAME_YIELD_GROUP_XNA',
 'NAME_CONTRACT_STATUS_XNA']

In [45]:
XAP_col

['NAME_CASH_LOAN_PURPOSE_XAP', 'CODE_REJECT_REASON_XAP']

In [47]:
XNA_XAP_col = XNA_col + XAP_col

In [48]:
XNA_XAP_col

['NAME_CONTRACT_TYPE_XNA',
 'NAME_CASH_LOAN_PURPOSE_XNA',
 'NAME_PAYMENT_TYPE_XNA',
 'CODE_REJECT_REASON_XNA',
 'NAME_CLIENT_TYPE_XNA',
 'NAME_GOODS_CATEGORY_XNA',
 'NAME_PORTFOLIO_XNA',
 'NAME_PRODUCT_TYPE_XNA',
 'NAME_SELLER_INDUSTRY_XNA',
 'NAME_YIELD_GROUP_XNA',
 'NAME_CONTRACT_STATUS_XNA',
 'NAME_CASH_LOAN_PURPOSE_XAP',
 'CODE_REJECT_REASON_XAP']

In [50]:
cat_features = [f for f in app_test.columns if app_test[f].dtype == 'object' and f not in XNA_XAP_col]

In [51]:
cat_features

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [52]:
num_features = [f for f in app_test.columns if app_test[f].dtype != 'object' and f not in XNA_XAP_col]

In [53]:
num_features

['SK_ID_CURR',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT_x',
 'AMT_ANNUITY_x',
 'AMT_GOODS_PRICE_x',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START_x',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_

In [54]:
df_cat = app_test[cat_features]

In [55]:
df_cat.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Married,House / apartment,NaN,TUESDAY,Kindergarten,NaN,block of flats,"Stone, brick",No
1,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Low-skill Laborers,FRIDAY,Self-employed,NaN,NaN,NaN,NaN
2,Cash loans,M,Y,Y,NaN,Working,Higher education,Married,House / apartment,Drivers,MONDAY,Transport: type 3,NaN,NaN,NaN,NaN
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Sales staff,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No
4,Cash loans,M,Y,N,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,NaN,FRIDAY,Business Entity Type 3,NaN,NaN,NaN,NaN


In [56]:
df_cat['SK_ID_CURR'] = app_test['SK_ID_CURR']

In [57]:
df_cat

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,SK_ID_CURR
0,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Married,House / apartment,NaN,TUESDAY,Kindergarten,NaN,block of flats,"Stone, brick",No,100001
1,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Low-skill Laborers,FRIDAY,Self-employed,NaN,NaN,NaN,NaN,100005
2,Cash loans,M,Y,Y,NaN,Working,Higher education,Married,House / apartment,Drivers,MONDAY,Transport: type 3,NaN,NaN,NaN,NaN,100013
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Sales staff,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No,100028
4,Cash loans,M,Y,N,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,NaN,FRIDAY,Business Entity Type 3,NaN,NaN,NaN,NaN,100038
5,Cash loans,F,Y,Y,Unaccompanied,State servant,Secondary / secondary special,Married,House / apartment,Drivers,MONDAY,Government,not specified,block of flats,Block,No,100042
6,Cash loans,M,Y,Y,Unaccompanied,Working,Higher education,Married,House / apartment,High skill tech staff,THURSDAY,Industry: type 9,NaN,NaN,NaN,NaN,100057
7,Cash loans,M,N,Y,Unaccompanied,Working,Higher education,Single / not married,With parents,Core staff,FRIDAY,Self-employed,NaN,NaN,NaN,NaN,100065
8,Cash loans,F,N,Y,Unaccompanied,State servant,Higher education,Married,House / apartment,Core staff,THURSDAY,School,NaN,block of flats,"Stone, brick",No,100066
9,Cash loans,F,Y,Y,Family,Working,Higher education,Civil marriage,House / apartment,Sales staff,TUESDAY,Trade: type 2,NaN,NaN,NaN,NaN,100067


In [58]:
df_num = app_test[num_features]

In [59]:
df_num

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329,-5170.0,...,0.0,0.0,31.0,1.0,0.0,0.0,0.0,0.0,110.0,30.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469,-9118.0,...,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458,-2175.0,...,0.0,0.0,79.0,7.0,0.0,0.0,0.0,0.0,103.0,41.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866,-2000.0,...,0.0,0.0,164.0,0.0,0.0,0.0,0.0,0.0,263.0,133.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191,-4000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,100042,0,270000.0,959688.0,34600.5,810000.0,0.025164,-18604,-12009,-6116.0,...,0.0,0.0,190.0,7.0,1.0,0.0,0.0,0.0,246.0,171.0
6,100057,2,180000.0,499221.0,22117.5,373500.0,0.022800,-16685,-2580,-10125.0,...,0.0,0.0,71.0,0.0,0.0,0.0,0.0,0.0,229.0,67.0
7,100065,0,166500.0,180000.0,14220.0,180000.0,0.005144,-9516,-1387,-5063.0,...,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,170.0,92.0
8,100066,0,315000.0,364896.0,28957.5,315000.0,0.046220,-12744,-1013,-1686.0,...,0.0,0.0,12.0,1.0,0.0,0.0,0.0,0.0,23.0,20.0
9,100067,1,162000.0,45000.0,5337.0,45000.0,0.018634,-10395,-2625,-8124.0,...,0.0,0.0,119.0,1.0,0.0,0.0,0.0,0.0,77.0,134.0


In [60]:
from sklearn.impute import SimpleImputer

# Most frequently occuring values imputation of missing categorical values
imputer_cat = SimpleImputer(strategy = 'most_frequent')

# Fit on the test data
imputer_cat.fit(df_cat)

# Transform testing data
test_cat = imputer_cat.transform(df_cat)

print('Testing data shape: ', test_cat.shape)

Testing data shape:  (48744, 17)


In [61]:
# Median values imputation of missing numerical values
imputer_num = SimpleImputer(strategy = 'median')

# Fit on the test data
imputer_num.fit(df_num)

# Transform testing data
test_num = imputer_num.transform(df_num)

print('Testing data shape: ', test_num.shape)

Testing data shape:  (48744, 345)


In [62]:
cat_features.append('SK_ID_CURR')

In [63]:
test_cat_df= pd.DataFrame(data = test_cat, columns = cat_features)

In [64]:
test_cat_df

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,SK_ID_CURR
0,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Married,House / apartment,Laborers,TUESDAY,Kindergarten,reg oper account,block of flats,"Stone, brick",No,100001
1,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Low-skill Laborers,FRIDAY,Self-employed,reg oper account,block of flats,Panel,No,100005
2,Cash loans,M,Y,Y,Unaccompanied,Working,Higher education,Married,House / apartment,Drivers,MONDAY,Transport: type 3,reg oper account,block of flats,Panel,No,100013
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Sales staff,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No,100028
4,Cash loans,M,Y,N,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Laborers,FRIDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No,100038
5,Cash loans,F,Y,Y,Unaccompanied,State servant,Secondary / secondary special,Married,House / apartment,Drivers,MONDAY,Government,not specified,block of flats,Block,No,100042
6,Cash loans,M,Y,Y,Unaccompanied,Working,Higher education,Married,House / apartment,High skill tech staff,THURSDAY,Industry: type 9,reg oper account,block of flats,Panel,No,100057
7,Cash loans,M,N,Y,Unaccompanied,Working,Higher education,Single / not married,With parents,Core staff,FRIDAY,Self-employed,reg oper account,block of flats,Panel,No,100065
8,Cash loans,F,N,Y,Unaccompanied,State servant,Higher education,Married,House / apartment,Core staff,THURSDAY,School,reg oper account,block of flats,"Stone, brick",No,100066
9,Cash loans,F,Y,Y,Family,Working,Higher education,Civil marriage,House / apartment,Sales staff,TUESDAY,Trade: type 2,reg oper account,block of flats,Panel,No,100067


In [65]:
test_num_df= pd.DataFrame(data = test_num, columns = num_features)

In [66]:
test_num_df

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
0,100001.0,0.0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241.0,-2329.0,-5170.0,...,0.0,0.0,31.0,1.0,0.0,0.0,0.0,0.0,110.0,30.0
1,100005.0,0.0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064.0,-4469.0,-9118.0,...,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0
2,100013.0,0.0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038.0,-4458.0,-2175.0,...,0.0,0.0,79.0,7.0,0.0,0.0,0.0,0.0,103.0,41.0
3,100028.0,2.0,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976.0,-1866.0,-2000.0,...,0.0,0.0,164.0,0.0,0.0,0.0,0.0,0.0,263.0,133.0
4,100038.0,1.0,180000.0,625500.0,32067.0,625500.0,0.010032,-13040.0,-2191.0,-4000.0,...,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,66.0,27.0
5,100042.0,0.0,270000.0,959688.0,34600.5,810000.0,0.025164,-18604.0,-12009.0,-6116.0,...,0.0,0.0,190.0,7.0,1.0,0.0,0.0,0.0,246.0,171.0
6,100057.0,2.0,180000.0,499221.0,22117.5,373500.0,0.022800,-16685.0,-2580.0,-10125.0,...,0.0,0.0,71.0,0.0,0.0,0.0,0.0,0.0,229.0,67.0
7,100065.0,0.0,166500.0,180000.0,14220.0,180000.0,0.005144,-9516.0,-1387.0,-5063.0,...,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,170.0,92.0
8,100066.0,0.0,315000.0,364896.0,28957.5,315000.0,0.046220,-12744.0,-1013.0,-1686.0,...,0.0,0.0,12.0,1.0,0.0,0.0,0.0,0.0,23.0,20.0
9,100067.0,1.0,162000.0,45000.0,5337.0,45000.0,0.018634,-10395.0,-2625.0,-8124.0,...,0.0,0.0,119.0,1.0,0.0,0.0,0.0,0.0,77.0,134.0


In [67]:
print(test_cat_df.shape)
print(test_num_df.shape)

(48744, 17)
(48744, 345)


In [68]:
test_cat_df.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,SK_ID_CURR
0,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Married,House / apartment,Laborers,TUESDAY,Kindergarten,reg oper account,block of flats,"Stone, brick",No,100001
1,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Low-skill Laborers,FRIDAY,Self-employed,reg oper account,block of flats,Panel,No,100005
2,Cash loans,M,Y,Y,Unaccompanied,Working,Higher education,Married,House / apartment,Drivers,MONDAY,Transport: type 3,reg oper account,block of flats,Panel,No,100013
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Sales staff,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No,100028
4,Cash loans,M,Y,N,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Laborers,FRIDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No,100038


In [69]:
test_num_df.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
0,100001.0,0.0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241.0,-2329.0,-5170.0,...,0.0,0.0,31.0,1.0,0.0,0.0,0.0,0.0,110.0,30.0
1,100005.0,0.0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064.0,-4469.0,-9118.0,...,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0
2,100013.0,0.0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038.0,-4458.0,-2175.0,...,0.0,0.0,79.0,7.0,0.0,0.0,0.0,0.0,103.0,41.0
3,100028.0,2.0,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976.0,-1866.0,-2000.0,...,0.0,0.0,164.0,0.0,0.0,0.0,0.0,0.0,263.0,133.0
4,100038.0,1.0,180000.0,625500.0,32067.0,625500.0,0.010032,-13040.0,-2191.0,-4000.0,...,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,66.0,27.0


In [70]:
test_num_df['SK_ID_CURR'] = test_num_df['SK_ID_CURR'].astype('object')

In [71]:
test_num_df

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
0,100001,0.0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241.0,-2329.0,-5170.0,...,0.0,0.0,31.0,1.0,0.0,0.0,0.0,0.0,110.0,30.0
1,100005,0.0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064.0,-4469.0,-9118.0,...,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0
2,100013,0.0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038.0,-4458.0,-2175.0,...,0.0,0.0,79.0,7.0,0.0,0.0,0.0,0.0,103.0,41.0
3,100028,2.0,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976.0,-1866.0,-2000.0,...,0.0,0.0,164.0,0.0,0.0,0.0,0.0,0.0,263.0,133.0
4,100038,1.0,180000.0,625500.0,32067.0,625500.0,0.010032,-13040.0,-2191.0,-4000.0,...,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,66.0,27.0
5,100042,0.0,270000.0,959688.0,34600.5,810000.0,0.025164,-18604.0,-12009.0,-6116.0,...,0.0,0.0,190.0,7.0,1.0,0.0,0.0,0.0,246.0,171.0
6,100057,2.0,180000.0,499221.0,22117.5,373500.0,0.022800,-16685.0,-2580.0,-10125.0,...,0.0,0.0,71.0,0.0,0.0,0.0,0.0,0.0,229.0,67.0
7,100065,0.0,166500.0,180000.0,14220.0,180000.0,0.005144,-9516.0,-1387.0,-5063.0,...,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,170.0,92.0
8,100066,0.0,315000.0,364896.0,28957.5,315000.0,0.046220,-12744.0,-1013.0,-1686.0,...,0.0,0.0,12.0,1.0,0.0,0.0,0.0,0.0,23.0,20.0
9,100067,1.0,162000.0,45000.0,5337.0,45000.0,0.018634,-10395.0,-2625.0,-8124.0,...,0.0,0.0,119.0,1.0,0.0,0.0,0.0,0.0,77.0,134.0


In [72]:
test_imputed_df = test_num_df.merge(test_cat_df, on = 'SK_ID_CURR')
test_imputed_df.shape

(48744, 361)

In [73]:
test_imputed_df

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,100001,0.0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241.0,-2329.0,-5170.0,...,Higher education,Married,House / apartment,Laborers,TUESDAY,Kindergarten,reg oper account,block of flats,"Stone, brick",No
1,100005,0.0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064.0,-4469.0,-9118.0,...,Secondary / secondary special,Married,House / apartment,Low-skill Laborers,FRIDAY,Self-employed,reg oper account,block of flats,Panel,No
2,100013,0.0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038.0,-4458.0,-2175.0,...,Higher education,Married,House / apartment,Drivers,MONDAY,Transport: type 3,reg oper account,block of flats,Panel,No
3,100028,2.0,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976.0,-1866.0,-2000.0,...,Secondary / secondary special,Married,House / apartment,Sales staff,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No
4,100038,1.0,180000.0,625500.0,32067.0,625500.0,0.010032,-13040.0,-2191.0,-4000.0,...,Secondary / secondary special,Married,House / apartment,Laborers,FRIDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No
5,100042,0.0,270000.0,959688.0,34600.5,810000.0,0.025164,-18604.0,-12009.0,-6116.0,...,Secondary / secondary special,Married,House / apartment,Drivers,MONDAY,Government,not specified,block of flats,Block,No
6,100057,2.0,180000.0,499221.0,22117.5,373500.0,0.022800,-16685.0,-2580.0,-10125.0,...,Higher education,Married,House / apartment,High skill tech staff,THURSDAY,Industry: type 9,reg oper account,block of flats,Panel,No
7,100065,0.0,166500.0,180000.0,14220.0,180000.0,0.005144,-9516.0,-1387.0,-5063.0,...,Higher education,Single / not married,With parents,Core staff,FRIDAY,Self-employed,reg oper account,block of flats,Panel,No
8,100066,0.0,315000.0,364896.0,28957.5,315000.0,0.046220,-12744.0,-1013.0,-1686.0,...,Higher education,Married,House / apartment,Core staff,THURSDAY,School,reg oper account,block of flats,"Stone, brick",No
9,100067,1.0,162000.0,45000.0,5337.0,45000.0,0.018634,-10395.0,-2625.0,-8124.0,...,Higher education,Civil marriage,House / apartment,Sales staff,TUESDAY,Trade: type 2,reg oper account,block of flats,Panel,No


In [80]:
test_imputed_df['SK_ID_CURR'] = test_imputed_df['SK_ID_CURR'].astype('int64')

In [81]:
test_imputed_df.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,100001,0.0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241.0,-2329.0,-5170.0,...,Higher education,Married,House / apartment,Laborers,TUESDAY,Kindergarten,reg oper account,block of flats,"Stone, brick",No
1,100005,0.0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064.0,-4469.0,-9118.0,...,Secondary / secondary special,Married,House / apartment,Low-skill Laborers,FRIDAY,Self-employed,reg oper account,block of flats,Panel,No
2,100013,0.0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038.0,-4458.0,-2175.0,...,Higher education,Married,House / apartment,Drivers,MONDAY,Transport: type 3,reg oper account,block of flats,Panel,No
3,100028,2.0,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976.0,-1866.0,-2000.0,...,Secondary / secondary special,Married,House / apartment,Sales staff,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No
4,100038,1.0,180000.0,625500.0,32067.0,625500.0,0.010032,-13040.0,-2191.0,-4000.0,...,Secondary / secondary special,Married,House / apartment,Laborers,FRIDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No


In [82]:
#Dummiying

test_final = pd.get_dummies(test_imputed_df, drop_first=True)

print('Test Features shape: ', test_final.shape)

Test Features shape:  (48744, 465)


In [83]:
test_final.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes
0,100001,0.0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241.0,-2329.0,-5170.0,...,0,0,0,0,0,0,0,1,0,0
1,100005,0.0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064.0,-4469.0,-9118.0,...,0,0,0,0,0,0,1,0,0,0
2,100013,0.0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038.0,-4458.0,-2175.0,...,0,0,0,0,0,0,1,0,0,0
3,100028,2.0,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976.0,-1866.0,-2000.0,...,0,0,0,0,0,0,1,0,0,0
4,100038,1.0,180000.0,625500.0,32067.0,625500.0,0.010032,-13040.0,-2191.0,-4000.0,...,0,0,0,0,0,0,1,0,0,0


In [84]:
test_final.to_csv('imputed_test.csv', index=False)